# Monterey Flipper Pinball Standings Projections

## Purpose
This notebook is used to project the final standings of the Monterey Flipper Pinball league.
The highest 5 weekly scores for each player are summed at the end of the season to produce a winner.
There are 10 weeks total, so you can drop up to 5 scores if you attend every week.

* As of right now it is the end of week 4, my initial idea is to take your average score and use that to replace any lower scores, as well as the correct number of future weeks.


## TODO/Issues
- Possibly redo converting the json to a dataframe, there has to be a better way
- Projections fail if the average is taken over a player's best games
- API fails to pull all players in Winter 17 MFP league, (series_id = 475)
- add 'score to beat' (the person's 5th best score)

In [329]:
import pandas as pd
import numpy as np
import requests
import json
from collections import defaultdict

In [330]:
# MatchPlay API with the Series ID for MFP Spring 18
series_id = '582'
series_url = 'https://matchplay.events/data/series/'+series_id+'/standings'

In [331]:
# Call the API
r = requests.get(series_url)
data = r.json()

In [332]:
# Split up the response, which gives overall standings and also weekly scores (tournament_points)
overall = data['overall']
overall = pd.DataFrame.from_records(overall)
tournament_points = data['tournament_points']

In [333]:
# Janky code to rearrange the tournament_points json response

points = defaultdict(list)
num_weeks = 0
for week in tournament_points:
    if week['status'] != 'planned':
        num_weeks += 1
        for player in week['points'].items():
            points[(player[0],week['tournament_id'])].append(player[1])


In [334]:
# Changing the points dict into a tidy dataframe

points_df = pd.DataFrame(points)
points_df = points_df.transpose().reset_index(1).pivot(columns='level_1',values=0)

In [335]:
# Url to grab weekly standings for names
week_ids = points_df.columns[:num_weeks]
tournament_url_a = 'https://matchplay.events/data/tournaments/'
tournament_url_b = '/standings'

In [336]:
player_names = {}
for week_id in week_ids:
    tournament_url = tournament_url_a + str(week_id) + tournament_url_b
    r = requests.get(tournament_url)
    data = r.json()
    for player in data:
        if player['player_id'] not in player_names:
            player_names[str(player['player_id'])] = player['name']

In [337]:
points_df.index = points_df.index.to_series().map(player_names)

In [338]:
# Apply function that takes the row and does necessary computations for extra columns
weeks_left = 10-num_weeks

def compute_projections(row):
    # Compute average score of all weeks
    row['avg_score'] = row[:num_weeks].mean()
    
    # Number of weeks played
    row['num_games'] = row[:num_weeks].count()
    
    # Total number of points accumulated
    row['cur_total'] = sum([points for points in row[:num_weeks] if points > 0])

    # Find best five games for a player
    best_games = [points for points in row[:num_weeks] if points > 0]
    best_games.sort(reverse=True)
    best_games = best_games[:5]
    
    # Find a player's current score, the metric by which players are judged at the end of the season
    row['cur_score'] = sum(best_games)
    
    # Compute average score of a player's best 5 weeks
    #row['avg_score'] = row['cur_score']/len(best_games)
    
    # Exclude a players
    best_games_aa = [points for points in best_games if points >= row['avg_score']]
    best_games_aa.sort(reverse=True)    
    
    # mean_adj_score: the points added to a player's projection to supplant their below average games
    # initialize variable to 0 for players that have 5 games above their average
    #     this may be deprecated by finding the average of the 5 best games instead of all games
    row['mean_adj_score'] = 0
    
    # If a player has five games above their average, no games are replaced by that average
    if len(best_games_aa) >= 5:
        row['kept_score'] = sum(best_games_aa[:5])
        row['proj_score'] = row['kept_score']
        
    # Replace weeks below average with the average score, depending on how many weeks are left
    if len(best_games_aa) < 5:
        row['kept_score'] = sum(best_games_aa)
        
        # replace the weeks below average to fill to 5, or however many weeks are left if it wouldn't be possible
        if (5-len(best_games_aa) <= weeks_left):
            row['mean_adj_score'] = (5-len(best_games_aa))*row['avg_score']
        else:
            row['kept_score'] = sum(best_games[:-weeks_left])
            row['mean_adj_score'] = weeks_left*row['avg_score']
        
        # Project final score based on player's average
        row['proj_score'] = row['kept_score'] + row['mean_adj_score']
    
    # Find a player's maximum possible score assuming they score a perfect 35 every remaining week
    if weeks_left < 5:
        row['max_score_possible'] = sum(best_games[:5-weeks_left]) + weeks_left*35
    else:
        row['max_score_possible'] = 5*35
    
    # The number of weeks of average score added to a player's projection,
    #   so the number of weeks the player would need to do above average to reach their projection
    row['remaining_weeks_above_avg'] = (row['mean_adj_score']/row['avg_score'])
    
    # The score a player needs to beat in a future week to improve their score
    #row['score_to_beat'] = best_games[4]
    row['score_to_beat'] = 0
    if len(best_games) == 5:
        row['score_to_beat'] = best_games[-1]
    
    
    return row


In [339]:
# Apply the calculations
final_points_df = points_df.apply(compute_projections, axis=1)

In [340]:
final_points_df.sort_values('proj_score', ascending = False, inplace=True)

In [341]:
current_leader = final_points_df.cur_score.max()
print current_leader

# calculate the average a player must get in the remaining games to beat the leader
def compute_required_avg(row):
    if row['max_score_possible'] <= current_leader:
        row['required_avg'] = 0
    elif row['cur_score'] == current_leader:
        row['required_avg'] = 0
    else:
        # Find best five games for a player
        best_games = [points for points in row[:num_weeks] if points > 0]
        best_games.sort(reverse=True)
        best_games = best_games[:5]
        row['required_avg'] = ((current_leader+1) - sum(best_games[:5-weeks_left]))/weeks_left
    
    return row

145.0


In [342]:
final_points_df = final_points_df.apply(compute_required_avg, axis=1)

In [343]:
final_points_df.sort_values('proj_score', ascending = False)

level_1,17082,17083,17084,17085,17086,17087,17088,17089,avg_score,num_games,cur_total,cur_score,mean_adj_score,kept_score,proj_score,max_score_possible,remaining_weeks_above_avg,score_to_beat,required_avg
Cary Carmichael,21.0,14.0,27.0,33.0,29.0,27.0,21.0,29.0,25.125000,8.0,201.0,145.0,0.000000,145.0,145.000000,161.0,0.0,27.0,0.0
Matthew Talley,21.0,27.0,27.0,NaN,23.0,29.0,27.0,27.0,25.857143,7.0,181.0,137.0,0.000000,137.0,137.000000,153.0,0.0,27.0,31.5
Tony Lavigna,25.0,19.0,NaN,17.0,NaN,25.0,29.0,33.0,24.666667,6.0,148.0,131.0,24.666667,112.0,136.666667,157.0,1.0,19.0,29.5
John Tracey,23.0,25.0,25.0,27.0,19.0,23.0,21.0,27.0,23.750000,8.0,190.0,127.0,23.750000,104.0,127.750000,149.0,1.0,23.0,33.5
Greg Friedman,33.0,17.0,NaN,23.0,25.0,NaN,15.0,NaN,22.600000,5.0,113.0,113.0,45.200000,81.0,126.200000,151.0,2.0,15.0,32.5
Mike Hubbard,NaN,27.0,20.0,22.0,21.0,31.0,19.0,19.0,22.714286,7.0,159.0,121.0,45.428571,80.0,125.428571,150.0,2.0,20.0,33.0
Fred Hamilton,21.0,18.0,27.0,25.0,21.0,23.0,27.0,21.0,22.875000,8.0,183.0,123.0,22.875000,102.0,124.875000,149.0,1.0,21.0,33.5
Lucy Cachux,15.0,27.0,25.0,25.0,18.0,18.0,17.0,25.0,21.250000,8.0,170.0,120.0,21.250000,102.0,123.250000,147.0,1.0,18.0,34.5
Ernie Deakyne,25.0,23.0,21.0,26.0,23.0,14.0,25.0,21.0,22.250000,8.0,178.0,122.0,0.000000,122.0,122.000000,146.0,0.0,23.0,35.0
Quinn Tomlinson,19.0,25.0,25.0,21.0,NaN,25.0,23.0,23.0,23.000000,7.0,161.0,121.0,0.000000,121.0,121.000000,145.0,0.0,23.0,0.0
